In [1]:
import gzip
import random
import csv
from collections import defaultdict
from sklearn import linear_model

In [2]:
path="trainInteractions.csv.gz"

In [3]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [4]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [5]:
dataset = list(readCSV(path))

In [6]:
dataset[0]

('88348277',
 '03969194',
 {'user_id': '88348277',
  'recipe_id': '03969194',
  'date': '2004-12-23',
  'rating': '5'})

2. Classification

In [7]:
import sklearn
from sklearn import linear_model

In [8]:
def build_train_set(dataset):
    itemsPerUser = defaultdict(set)
    usersPerItem=defaultdict(set)
    itemSet=set([d[1] for d in dataset])
    for d in dataset:
        user,item = d[0], d[1]
        itemsPerUser[user].add(item)
        usersPerItem[item].add(user)
    train_set=[]
    random.seed(50)
    for d in dataset:
        positive_entry=[d[0],d[1],1]
        negative_entry_item_set=itemSet.difference(itemsPerUser[d[0]])
        for item in list(negative_entry_item_set):
            negative_entry=[d[0],item,0]
            train_set.append(negative_entry)
        train_set.append(positive_entry)
    return train_set

In [ ]:
train_set=build_train_set(dataset)

In [ ]:
itemsPerUser = defaultdict(set)
usersPerItem=defaultdict(set)
for d in train_set:
    user,item = d[0], d[1]
    itemsPerUser[user].add(item)
    usersPerItem[item].add(user)

itemSet=set([d[1] for d in train_set])
userSet=set([d[0] for d in train_set])

In [ ]:
# calculate most popular set in train_set
recipeCount = defaultdict(int)
totalCooked = 0
for d in train_set:
    recipeCount[d[1]] += 1
    totalCooked += 1

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalCooked*0.6: break

In [ ]:
def feature(user, item):
    feat = [1]
    if item in return1:
        feat.append(1)
    else:
        feat.append(0)    
    maxSim=0
    for d in itemsPerUser[user]:
        sim=Jaccard(usersPerItem[d],usersPerItem[item])
        maxSim=max(maxSim,sim)
    feat.append(maxSim-0.01)
    return feat

In [ ]:
X=[]
y=[]

In [ ]:
for d in train_set:
    user=d[0]
    item=d[1]
    X.append(feature(user,item))
    y.append(d[2]==1)

In [ ]:
model = sklearn.linear_model.LogisticRegression()
model.fit(X, y)

In [ ]:
X_test=[]

In [ ]:
predictions = open("predictions_Made.txt", 'w')
for l in open("stub_Made.txt"):
    if l.startswith("user_id"):
        predictions.write(l)
        continue
    user,item = l.strip().split('-')
    X_test.append(feature(user,item))
y_predict=model.predict(X_test)

i=0
for l in open("stub_Made.txt"):
    if l.startswith("user_id"):
        continue
    user,item = l.strip().split('-')
    predict=y_predict[i]
    if predict==True:
        predictions.write(user + '-' + item + ",1\n")
    else:
        predictions.write(user + '-' + item + ",0\n")
    i+=1
predictions.close()
print("predicting finished!")